In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
# Ethical data
%cd /content/drive/MyDrive/NLP_project/data

/content/drive/MyDrive/NLP_project/data


In [75]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import random

In [76]:
os.getcwd()

'/content/drive/MyDrive/NLP_project/data'

In [77]:
# Get a list of JSON files
files = os.listdir()
files = sorted(files)
# Read each JSON file and save the data
data = []
for file in files:
  if file.endswith('.json'):
    if file == 'data4.json':
      continue
    with open(file, 'r') as f:
      data.append(json.load(f))

data_test = []
with open('data4.json', 'r') as f:
  data_test.append(json.load(f))

In [78]:
def createDF(x):
  # Create a DataFrame
  df_list = []
  previous_id = None
  # Select only the speaker, text, types and is_immoral var for each sentence
  # Space break by id
  for data_num in x:
    for record in data_num:
        id = record['id']
        for sentence in record['sentences']:
            speaker = 'User' + str(sentence['speaker'])
            text = sentence['text']
            text = text.replace('\t', ' ') # Remove tap in text

            if len(sentence['types'])==1:
              types = sentence['types']
            else:   # Multi-label
              types = random.choices(sentence['types'])

            is_immoral = sentence['is_immoral']
            df_list.append([id,speaker, text, types, is_immoral])
        
        previous_id = id


  df = pd.DataFrame(df_list, columns=['ID','Speaker', 'Text', 'Type','Is_Immoral'])
  return df

df = createDF(data)
test = createDF(data_test)


In [79]:
def get_most_frequent_type(x):
    return x.mode()[0]

df_type = df.groupby('ID')['Type'].agg(get_most_frequent_type).reset_index()
df_type.columns = ['ID', 'Type']



df_type['Type'] = df_type['Type'].astype(str)
df_sample1 = df_type[df_type['Type'] == "['IMMORAL_NONE']"].sample(19031)
df_sample2 = df_type[df_type['Type'] == "['CENSURE']"].sample(21269)

In [80]:
df_filtered = df[~df['ID'].isin(df_sample1['ID']) & ~df['ID'].isin(df_sample2['ID'])]
df = df_filtered

In [55]:

#df_filtered['Type'].value_counts()


[IMMORAL_NONE]      52763
[CENSURE]           13518
[HATE]               2928
[DISCRIMINATION]     1466
[ABUSE]              1414
[SEXUAL]             1361
[VIOLENCE]           1307
[CRIME]               391
Name: Type, dtype: int64

In [81]:
def get_most_frequent_type(x):
    return x.mode()[0]

df_type = df_filtered.groupby('ID')['Type'].agg(get_most_frequent_type).reset_index()
df_type.columns = ['ID', 'Type']


In [67]:
df_type['Type'].value_counts()

[CENSURE]           5030
[IMMORAL_NONE]      4999
[HATE]              2836
[DISCRIMINATION]    2218
[ABUSE]             1755
[SEXUAL]            1102
[VIOLENCE]           414
[CRIME]              398
Name: Type, dtype: int64

In [82]:
# Create Train, Validation, Test sets
# Get a unique value by ID
unique_ids = df['ID'].dropna().unique()

# divide ID by train, valid: 3:1
train_ids, valid_ids = train_test_split(unique_ids, test_size=1/3, random_state=42)


# assign a row for each ID
train = df[df['ID'].isin(train_ids)]
valid = df[df['ID'].isin(valid_ids)]

del df, train_ids, valid_ids, unique_ids

In [83]:
def export_df_to_txt(df, filename):
# export_ dataframe to txt
    prev_id = None
    with open(filename, 'w') as f:
        f.write('\t'.join(df.columns) + '\n')
        for _, row in df.iterrows():
            if prev_id != row['ID']:
                f.write('\n')
            line = '\t'.join([str(item) for item in row.values])
            f.write(line + '\n')
            prev_id = row['ID']

In [84]:
# Call the function for each dataframe
export_df_to_txt(train, 'train.txt')
export_df_to_txt(valid, 'valid.txt')
export_df_to_txt(test, 'test.txt')